In [1]:
# from load_data import import_data
import pandas as pd
# First we sum the number of encounters and loiterings for each carrier.
# Then make a classification: if there are more encounters than loiterings, the carrier is not suspicious, if there is more loitering, the carrier is suspicious. (Don't hard-code the cutoff. We may want to do 66% loitering and 33% encounters in the future instead of 50-50)
# For every carrier in the carriers table, label a value 1 (suspicious) or 0 (not suspicious).
import merge



In [3]:
print(" > Importing local data...")
loitering = pd.read_csv('loitering.csv')
encounters = pd.read_csv('encounter.csv')
port_visits = pd.read_csv('port.csv')
def call_api():
    '''
    Load all new global fishing watch data since last API call
    '''
    path = 'api_data.xlsx'
    xls = pd.ExcelFile(path)
    dfList = []
    for sheet in ['carriers', 'encounter', 'loitering', 'carrier_port_visit']:
        dfList.append(pd.read_excel(path, sheet_name=sheet))
    return dfList
## call new data from api
print(" > Importing API data")
carriers_new, encounters_new, loitering_new, port_visits_new = call_api()
print(loitering.columns)
print(loitering_new.columns)

 > Importing local data...
 > Importing API data
Index(['id', 'type', 'start', 'end', 'lat', 'lon', 'vessel.id', 'vessel.type',
       'vessel.mmsi', 'vessel.name', 'vessel.flag',
       'vessel.origin_port.country', 'vessel.origin_port.name',
       'vessel.destination_port.country', 'vessel.destination_port.name',
       'vessel.authorizations.authorized', 'vessel.authorizations.unknown',
       'median_speed_knots', 'elevation_m', 'distance_from_shore_m',
       'distance_from_port_m', 'regions.rfmo', 'encounter.median_distance_km',
       'encounter.authorization_status', 'encounter.encountered_vessel.id',
       'encounter.encountered_vessel.type',
       'encounter.encountered_vessel.mmsi',
       'encounter.encountered_vessel.name',
       'encounter.encountered_vessel.flag',
       'encounter.encountered_vessel.origin_port.country',
       'encounter.encountered_vessel.origin_port.name',
       'encounter.encountered_vessel.destination_port.country',
       'encounter.encounter

In [4]:
(loitering['id'].isin(loitering_new['event_id']).sum())/loitering['id'].count()

0.999048160549203

In [5]:
(encounters['id'].isin(encounters_new['event_id']).sum())

0

In [190]:
# classified_data.head()

In [191]:
# classified_data['label'].sum()

In [192]:
# classified_data['label'].count()

In [193]:
# a = change_cutoff(classified_data, 0.99)

In [194]:
# a['label'].sum()

In [195]:
# carriers

In [ ]:
# num_encounter2 = []
# num_loitering2 = []
# for i in carriers.id:
# #     num_encounter2.append(encounter2[encounter2['vessel.id']==i].shape[0] + encounter2[encounter2['encounter.encountered_vessel.id']==i].shape[0])
#     num_loitering2.append(loitering2[loitering2['vessel.id']==i].shape[0])

# # append the num info into new column in carrier table
# carrier = carriers.assign(encounter2 = num_encounter2, loitering2 = num_loitering2)

In [ ]:
# carrier

In [53]:
encounters['start']

0       2015-02-19 13:20:00
1       2015-02-17 15:30:00
2       2015-02-15 17:40:00
3       2015-05-30 00:00:00
4       2015-02-02 20:10:00
                ...        
57802   2019-06-06 05:00:00
57803   2019-08-23 04:20:00
57804   2019-06-08 06:50:00
57805   2019-08-22 00:30:00
57806   2019-08-22 00:00:00
Name: start, Length: 57807, dtype: datetime64[ns]

# Encounter Merging

In [6]:
encounters_new.columns

Index(['event_id', 'event_type', 'event_start', 'event_end', 'event_lat',
       'event_lon', 'mpa', 'eez', 'rfmo', 'fao', 'major_fao', 'eez12nm',
       'boundingBox', 'startDistanceFromShoreKm', 'endDistanceFromShoreKm',
       'startDistanceFromPortKm', 'endDistanceFromPortKm', 'vessel1_id',
       'vessel1_flag', 'vessel1_name', 'vessel1_type', 'vessel1_ssvid',
       'vessel2_id', 'vessel2_flag', 'vessel2_name', 'vessel2_type',
       'vessel2_ssvid', 'medianDistanceKilometers', 'medianSpeedKnots',
       'encounter_type'],
      dtype='object')

In [7]:
encounters.columns

Index(['id', 'type', 'start', 'end', 'lat', 'lon', 'vessel.id', 'vessel.type',
       'vessel.mmsi', 'vessel.name', 'vessel.flag',
       'vessel.origin_port.country', 'vessel.origin_port.name',
       'vessel.destination_port.country', 'vessel.destination_port.name',
       'vessel.authorizations.authorized', 'vessel.authorizations.unknown',
       'median_speed_knots', 'elevation_m', 'distance_from_shore_m',
       'distance_from_port_m', 'regions.rfmo', 'encounter.median_distance_km',
       'encounter.authorization_status', 'encounter.encountered_vessel.id',
       'encounter.encountered_vessel.type',
       'encounter.encountered_vessel.mmsi',
       'encounter.encountered_vessel.name',
       'encounter.encountered_vessel.flag',
       'encounter.encountered_vessel.origin_port.country',
       'encounter.encountered_vessel.origin_port.name',
       'encounter.encountered_vessel.destination_port.country',
       'encounter.encountered_vessel.destination_port.name',
       'encount

In [18]:
encounters_new.columns = ['id', 'type', 'start', 'end', 'lat', 'lon','mpa', 'eez', 'regions.rfmo','fao', 'major_fao', 'eez12nm',
       'boundingBox', 'startDistanceFromShoreKm', 'endDistanceFromShoreKm',
       'startDistanceFromPortKm', 'endDistanceFromPortKm','vessel.id','vessel.flag','vessel.name','vessel.type','vessel.mmsi',
                  'encounter.encountered_vessel.id', 'encounter.encountered_vessel.flag', 'encounter.encountered_vessel.name',
                    'encounter.encountered_vessel.type','encounter.encountered_vessel.mmsi','medianDistanceKilometers', 'medianSpeedKnots',
       'encounter_type']

In [22]:
encounters_new.columns = ['event_id', 'event_type', 'event_start', 'event_end', 'event_lat',
       'event_lon', 'mpa', 'eez', 'rfmo', 'fao', 'major_fao', 'eez12nm',
       'boundingBox', 'startDistanceFromShoreKm', 'endDistanceFromShoreKm',
       'startDistanceFromPortKm', 'endDistanceFromPortKm', 'vessel1_id',
       'vessel1_flag', 'vessel1_name', 'vessel1_type', 'vessel1_ssvid',
       'vessel2_id', 'vessel2_flag', 'vessel2_name', 'vessel2_type',
       'vessel2_ssvid', 'medianDistanceKilometers', 'medianSpeedKnots',
       'encounter_type']

In [26]:
encounters_new = encounters_new.rename({
    'event_id': 'id',
    'event_type': 'type',
    'event_start': 'start',
    'event_end': 'end',
    'event_lat': 'lat',
    'event_lon': 'lon',
    'rfmo': 'regions.rfmo',
    'vessel1_id': 'vessel.id',
    'vessel1_flag': 'vessel.flag',
    'vessel1_name': 'vessel.name',
    'vessel1_type': 'vessel.type',
    'vessel1_ssvid': 'vessel.mmsi',
    'vessel2_id': 'encounter.encountered_vessel.id',
    'vessel2_flag': 'encounter.encountered_vessel.flag',
    'vessel2_name': 'encounter.encountered_vessel.name',
    'vessel2_type': 'encounter.encountered_vessel.type',
    'vessel2_ssvid': 'encounter.encountered_vessel.mmsi',
    },
    axis = 1)

In [27]:
encounters_new.columns

Index(['id', 'type', 'start', 'end', 'lat', 'lon', 'mpa', 'eez',
       'regions.rfmo', 'fao', 'major_fao', 'eez12nm', 'boundingBox',
       'startDistanceFromShoreKm', 'endDistanceFromShoreKm',
       'startDistanceFromPortKm', 'endDistanceFromPortKm', 'vessel.id',
       'vessel.flag', 'vessel.name', 'vessel.type', 'vessel.mmsi',
       'encounter.encountered_vessel.id', 'encounter.encountered_vessel.flag',
       'encounter.encountered_vessel.name',
       'encounter.encountered_vessel.type',
       'encounter.encountered_vessel.mmsi', 'medianDistanceKilometers',
       'medianSpeedKnots', 'encounter_type'],
      dtype='object')

In [186]:
# (encounter['start'].isin(encounter2['start'])).sum()

In [12]:
encounter = encounters_new[encounters_new['encounter_type']=='carrier-fishing'].copy()
encounter2 = encounters.copy()
encounter2['start'] = pd.to_datetime(encounters['start'].str[:-4])
encounter2['end'] = pd.to_datetime(encounters['end'].str[:-4])
(encounter['encounter.encountered_vessel.id'].isin(encounter2['encounter.encountered_vessel.id'])).sum()
(encounter['vessel.id'].isin(encounter2['encounter.encountered_vessel.id'])).sum()

dfdf = encounter[~((encounter['start'].isin(encounter2['start'])) & (encounter['vessel.id'].isin(encounter2['vessel.id'])) & (encounter['encounter.encountered_vessel.id'].isin(encounter2['encounter.encountered_vessel.id'])))]
dfdf = dfdf[~((dfdf['start'].isin(encounter2['start'])) & (dfdf['vessel.id'].isin(encounter2['encounter.encountered_vessel.id'])) & (dfdf['vessel.id'].isin(encounter2['encounter.encountered_vessel.id'])))]
dfdf['id'].count()

1534

In [13]:
encounters['start']

0        2015-02-19 13:20:00 UTC
1        2015-02-17 15:30:00 UTC
2        2015-02-15 17:40:00 UTC
3        2015-05-30 00:00:00 UTC
4        2015-02-02 20:10:00 UTC
                  ...           
57802    2019-06-06 05:00:00 UTC
57803    2019-08-23 04:20:00 UTC
57804    2019-06-08 06:50:00 UTC
57805    2019-08-22 00:30:00 UTC
57806    2019-08-22 00:00:00 UTC
Name: start, Length: 57807, dtype: object

In [97]:
frames = [encounter2,dfdf]

result = pd.concat(frames)

In [ ]:
compression_opts2 = dict(method='zip',
                        archive_name='new_encounter.csv')  
result.to_csv('new_encounter.zip', index=False,
          compression=compression_opts2)  

# Loitering Merging

In [14]:
loitering.columns

Index(['id', 'type', 'start', 'end', 'lat', 'lon', 'vessel.id', 'vessel.type',
       'vessel.mmsi', 'vessel.name', 'vessel.flag',
       'vessel.origin_port.country', 'vessel.origin_port.name',
       'vessel.destination_port.country', 'vessel.destination_port.name',
       'vessel.authorizations.authorized', 'vessel.authorizations.unknown',
       'median_speed_knots', 'elevation_m', 'distance_from_shore_m',
       'distance_from_port_m', 'regions.rfmo', 'encounter.median_distance_km',
       'encounter.authorization_status', 'encounter.encountered_vessel.id',
       'encounter.encountered_vessel.type',
       'encounter.encountered_vessel.mmsi',
       'encounter.encountered_vessel.name',
       'encounter.encountered_vessel.flag',
       'encounter.encountered_vessel.origin_port.country',
       'encounter.encountered_vessel.origin_port.name',
       'encounter.encountered_vessel.destination_port.country',
       'encounter.encountered_vessel.destination_port.name',
       'encount

In [15]:
loitering_new.columns

Index(['event_id', 'event_type', 'event_start', 'event_end', 'event_lat',
       'event_lon', 'mpa', 'eez', 'rfmo', 'fao', 'major_fao', 'eez12nm',
       'boundingBox', 'startDistanceFromShoreKm', 'endDistanceFromShoreKm',
       'startDistanceFromPortKm', 'endDistanceFromPortKm', 'vessel_id', 'flag',
       'vessel_name', 'vessel_type', 'vessel_ssvid', 'totalTimeHours',
       'totalDistanceKm', 'averageSpeedKnots', 'averageDistanceFromShoreKm'],
      dtype='object')

In [16]:
loitering_new.columns = ['id', 'type', 'start', 'end', 'lat', 'lon','mpa', 'eez', 'regions.rfmo','fao', 'major_fao', 'eez12nm',
       'boundingBox', 'startDistanceFromShoreKm', 'endDistanceFromShoreKm',
       'startDistanceFromPortKm', 'endDistanceFromPortKm','vessel.id','vessel.flag','vessel.name','vessel.type','vessel.mmsi',
                     'totalTimeHours',
       'totalDistanceKm', 'averageSpeedKnots', 'averageDistanceFromShoreKm']

In [29]:
loitering_new = loitering_new.rename({
    'event_id': 'id',
    'event_type': 'type',
    'event_start': 'start',
    'event_end': 'end',
    'event_lat': 'lat',
    'event_lon': 'lon',
    'rfmo': 'regions.rfmo',
    'vessel_id': 'vessel.id',
    'vessel_flag': 'vessel.flag',
    'vessel_name': 'vessel.name',
    'vessel_type': 'vessel.type',
    'vessel_ssvid': 'vessel.mmsi',
    },
    axis = 1)

,id,type,start,end,lat,lon,mpa,eez,regions.rfmo,fao,...,endDistanceFromPortKm,vessel.id,vessel.flag,vessel.name,vessel.type,vessel.mmsi,totalTimeHours,totalDistanceKm,averageSpeedKnots,averageDistanceFromShoreKm
0,1faa8251c712f95a8e9c3eea4b1c5315,loitering,2012-01-01 02:20:52,2012-01-02 06:12:49,55.8269,147.1230,NaN,5690,"IWC, PICES, ACAP",61,...,392.129844,76bb4a3df-f818-cc8c-98de-375ac5d0eb3f,RUS,NaN,carrier,273511500,27.865833,31.519519,0.610754,318.687301
1,cbf0e222958e2d7457d619c003fb599e,loitering,2012-01-01 05:31:46,2012-01-05 01:47:23,55.9436,148.1235,NaN,5690,"PICES, IWC, ACAP",61,...,410.496188,8bf186cbe-e162-7bce-f761-b796dd85b6e5,RUS,NaN,carrier,273432530,92.260278,159.986263,0.936326,351.730632
2,7372e733f5c586a54cb0a487cf37ac35,loitering,2012-01-01 05:32:30,2012-01-03 03:39:28,56.8875,150.6353,NaN,5690,"IWC, PICES, ACAP",61,...,288.111219,392d412cc-cd54-fb89-26bd-b69fccb75b8e,RUS,NaN,carrier,273420240,46.116111,63.388497,0.742193,234.546243
3,f3f6979e9a441a8e8a6c1e8d452ed891,loitering,2012-01-01 05:32:55,2012-01-04 04:18:15,56.3073,145.1701,NaN,5690,"IWC, ACAP, PICES",61,...,296.779406,8e2091703-3571-eed0-b4d2-e198537ca9c4,RUS,NaN,carrier,273257000,70.755556,126.187022,0.962971,251.336207
4,411a3d66ebf847929dc45c709c53b410,loitering,2012-01-01 07:07:07,2012-01-05 01:46:13,56.5621,145.5512,NaN,5690,"PICES, IWC, ACAP",61,...,331.371625,1454a189f-ff76-cd53-68ef-1ed3d880fe86,RUS,NaN,carrier,273348810,90.651667,82.146169,0.489295,281.479910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282785,bd001233a7cdcb4747bf662f75981585,loitering,2022-10-30 21:51:15,2022-10-30 23:47:15,45.7018,152.9738,NaN,5690,"PICES, ACAP, WCPFC, IWC",61,...,386.181281,b2b74694c-c852-1831-2e0d-8c9c09731aa4,CHN,FU YUAN YU YUN 67,carrier,412440514,1.933333,6.229911,1.739939,149.927546
282786,a4972b663f5d0d1df3df9f894d05c66f,loitering,2022-10-30 21:55:41,2022-10-30 22:42:34,42.6999,145.8282,NaN,8487,"PICES, ACAP, IWC, WCPFC",61,...,68.447953,8e796d57a-aee9-7d94-aa4e-6e6bec461f48,JPN,NISSHIN MARU,carrier,431683000,0.781389,2.574595,1.779101,57.803584
282787,2bf338615457d458dd099dabb7cfa3f6,loitering,2022-10-30 22:43:11,2022-10-30 23:55:35,41.9735,151.9104,NaN,NaN,"WCPFC, IWC, PICES, ACAP, NPFC, NPAFC",61,...,463.458625,b7baecf62-25e9-83ce-e29c-66c986af7ab2,CHN,MING FA,carrier,412420829,1.206667,4.365326,1.953387,444.142745
282788,26148bf328a8abb264af4a4a309a3e8f,loitering,2022-10-30 22:59:07,2022-10-30 23:59:22,-2.8073,-148.8764,NaN,NaN,"ACAP, IATTC, IWC, SPRFMO",77,...,1093.256375,4e3401e92-242e-6104-da37-56b33f3aa7b2,KOR,AGNES 83,fishing,441043000,1.004167,3.093770,1.663570,682.335443


In [17]:
loitering_new[~loitering_new['id'].isin(loitering['id'])]

,id,type,start,end,lat,lon,mpa,eez,regions.rfmo,fao,...,endDistanceFromPortKm,vessel.id,vessel.flag,vessel.name,vessel.type,vessel.mmsi,totalTimeHours,totalDistanceKm,averageSpeedKnots,averageDistanceFromShoreKm
5,b930f9aff14ee86ccb4a17e572135777,loitering,2012-01-01 11:03:17,2012-01-02 06:54:14,-45.4244,-60.5720,NaN,NaN,"IWC, ACAP, ICCAT, CCSBT","41.3.1, 41, 41.3",...,406.890562,2be59719f-f9c9-0a91-3ccb-cbc7946e17b3,KOR,NaN,other_non_fishing,440046000,19.849167,7.057986,0.191998,382.002685
29,039de1cf0f1a9dface0e36554e1215f0,loitering,2012-01-02 06:09:13,2012-01-02 06:11:03,55.9446,147.0049,NaN,5690,"PICES, ACAP, IWC",61,...,391.400906,03a7d2fbd-d3a3-4ce9-f1d2-94f568abb7b4,RUS,NaN,carrier,273897100,0.030556,0.056114,0.991614,321.000000
33,88c66fa274bce76684e287f3da7eb099,loitering,2012-01-02 07:11:00,2012-01-02 22:07:21,-23.3481,12.8786,NaN,8395,"CCSBT, ICCAT, ACAP, IWC","47, 47.1.4, 47.1",...,177.556328,d81245d01-1c1c-571d-3e46-2c92798d89e2,PAN,ROMULUS,other_non_fishing,370136000,14.939167,35.838283,1.295328,162.992568
64,baf1393d1d72c5bff5195b80d3e9deb3,loitering,2012-01-03 00:13:40,2012-01-03 00:22:40,4.0767,-0.7362,NaN,8400,"IWC, ACAP, ICCAT, FCWC, NAMMCO","34, 34.3, 34.3.4",...,135.444391,9062663cb-b043-5cca-14b0-59ca43055c8a,PAN,CANARIAN REEFER,carrier,353585000,0.150000,0.340070,1.224152,126.000000
81,1190d0a5566fc2cf837e8d6c7c213852,loitering,2012-01-03 07:33:40,2012-01-04 03:23:45,-45.4592,-60.5638,NaN,NaN,"CCSBT, ACAP, IWC, ICCAT","41, 41.3.1, 41.3",...,407.572812,2be59719f-f9c9-0a91-3ccb-cbc7946e17b3,KOR,NaN,other_non_fishing,440046000,19.834722,5.273280,0.143553,385.013346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282785,bd001233a7cdcb4747bf662f75981585,loitering,2022-10-30 21:51:15,2022-10-30 23:47:15,45.7018,152.9738,NaN,5690,"PICES, ACAP, WCPFC, IWC",61,...,386.181281,b2b74694c-c852-1831-2e0d-8c9c09731aa4,CHN,FU YUAN YU YUN 67,carrier,412440514,1.933333,6.229911,1.739939,149.927546
282786,a4972b663f5d0d1df3df9f894d05c66f,loitering,2022-10-30 21:55:41,2022-10-30 22:42:34,42.6999,145.8282,NaN,8487,"PICES, ACAP, IWC, WCPFC",61,...,68.447953,8e796d57a-aee9-7d94-aa4e-6e6bec461f48,JPN,NISSHIN MARU,carrier,431683000,0.781389,2.574595,1.779101,57.803584
282787,2bf338615457d458dd099dabb7cfa3f6,loitering,2022-10-30 22:43:11,2022-10-30 23:55:35,41.9735,151.9104,NaN,NaN,"WCPFC, IWC, PICES, ACAP, NPFC, NPAFC",61,...,463.458625,b7baecf62-25e9-83ce-e29c-66c986af7ab2,CHN,MING FA,carrier,412420829,1.206667,4.365326,1.953387,444.142745
282788,26148bf328a8abb264af4a4a309a3e8f,loitering,2022-10-30 22:59:07,2022-10-30 23:59:22,-2.8073,-148.8764,NaN,NaN,"ACAP, IATTC, IWC, SPRFMO",77,...,1093.256375,4e3401e92-242e-6104-da37-56b33f3aa7b2,KOR,AGNES 83,fishing,441043000,1.004167,3.093770,1.663570,682.335443


In [63]:
# frames2 = [loitering2,loitering]

# result2 = pd.concat(frames2)

In [64]:
# result2.shape

(517577, 55)

In [197]:
# compression_opts = dict(method='zip',
#                         archive_name='loitering.csv')  
# result2.to_csv('loitering.zip', index=False,
#           compression=compression_opts)  

In [198]:
# loitering[~loitering['id'].isin(loitering2['id'])]

In [199]:
# loitering2['id'].count()

In [66]:
frames3 = [loitering,loitering_new[~loitering_new['id'].isin(loitering['id'])]]

result3 = pd.concat(frames3)
result3

,id,type,start,end,lat,lon,vessel.id,vessel.type,vessel.mmsi,vessel.name,...,eez12nm,boundingBox,startDistanceFromShoreKm,endDistanceFromShoreKm,startDistanceFromPortKm,endDistanceFromPortKm,totalTimeHours,totalDistanceKm,averageSpeedKnots,averageDistanceFromShoreKm
0,08373fbdc30976511c4fd237d1c796f6,loitering,2013-01-07 13:03:14 UTC,2013-01-08 01:45:25 UTC,-12.088447,61.467927,035a653cf-fb0c-a148-4f7b-8dfaf2a806f8,carrier,416308000,CHEN YU NO.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3f172225314fbf5c243c926706ff15d8,loitering,2013-01-07 02:00:14 UTC,2013-01-07 06:48:33 UTC,-12.404810,60.455511,035a653cf-fb0c-a148-4f7b-8dfaf2a806f8,carrier,416308000,CHEN YU NO.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8ada58fe6b417c0919a922f11f23efc1,loitering,2013-01-07 21:55:01 UTC,2013-01-07 23:36:12 UTC,-8.499583,-6.014750,036391f88-8a1b-98ed-c82d-d5871e837971,carrier,577106000,HARIMA 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e76f1af183ff7a32b0bf0f52e3cfbec6,loitering,2013-01-07 01:55:16 UTC,2013-01-07 06:52:15 UTC,-12.256155,61.902765,1512ca283-37b5-2737-87b9-047cf0f30433,carrier,352894000,TUNA QUEEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3816185f2d8931b10b2d2ef818519eb6,loitering,2013-01-07 23:53:05 UTC,2013-01-08 06:26:35 UTC,-12.262177,60.196520,1512ca283-37b5-2737-87b9-047cf0f30433,carrier,352894000,TUNA QUEEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282785,bd001233a7cdcb4747bf662f75981585,loitering,2022-10-30 21:51:15,2022-10-30 23:47:15,45.701800,152.973800,b2b74694c-c852-1831-2e0d-8c9c09731aa4,carrier,412440514,FU YUAN YU YUN 67,...,NaN,"153.008805, 45.7154766666667, 152.938895, 45.6...",150.0,149.0,391.855688,386.181281,1.933333,6.229911,1.739939,149.927546
282786,a4972b663f5d0d1df3df9f894d05c66f,loitering,2022-10-30 21:55:41,2022-10-30 22:42:34,42.699900,145.828200,8e796d57a-aee9-7d94-aa4e-6e6bec461f48,carrier,431683000,NISSHIN MARU,...,NaN,"145.833583333333, 42.7084166666667, 145.822883...",57.0,59.0,66.562430,68.447953,0.781389,2.574595,1.779101,57.803584
282787,2bf338615457d458dd099dabb7cfa3f6,loitering,2022-10-30 22:43:11,2022-10-30 23:55:35,41.973500,151.910400,b7baecf62-25e9-83ce-e29c-66c986af7ab2,carrier,412420829,MING FA,...,NaN,"151.93296, 41.9836266666667, 151.887746666667,...",444.0,445.0,465.287000,463.458625,1.206667,4.365326,1.953387,444.142745
282788,26148bf328a8abb264af4a4a309a3e8f,loitering,2022-10-30 22:59:07,2022-10-30 23:59:22,-2.807300,-148.876400,4e3401e92-242e-6104-da37-56b33f3aa7b2,fishing,441043000,AGNES 83,...,NaN,"-148.862948333333, -2.80338666666667, -148.889...",684.0,681.0,1095.637625,1093.256375,1.004167,3.093770,1.663570,682.335443


In [184]:
compression_opts0 = dict(method='zip',
                        archive_name='new_loitering.csv')  
result3.to_csv('new_loitering.zip', index=False,
          compression=compression_opts0)

# Visit merging

In [67]:
port_visits_new

,event_id,event_type,event_start,event_end,event_lat,event_lon,mpa,eez,rfmo,fao,...,top_destination1,anchorage_id2,at_dock2,distance_from_shore_km2,flag2,id2,lat2,lon2,name2,top_destination2
0,578adb3c2b6699cada7eaa0eda6db764,port_visit,2012-01-01 09:33:40,2012-01-06 00:51:01,56.1733,162.6062,NaN,5690,"PICES, ACAP, IWC",61,...,UST KAMCHATSK,5906e319,False,3.0,RUS,rus-ustkamchatsk,56.190914,162.481146,UST KAMCHATSK,UST KAMCHATSK
1,6d61c959b68561a9278d31bec7128bfb,port_visit,2012-01-01 13:59:29,2012-01-02 13:13:03,26.2559,120.1111,NaN,8486,"ACAP, WCPFC, IWC, APFIC",61,...,XIANG SHAN,3452fc4d,False,9.0,CHN,chn-zhoushan,29.609812,122.307929,ZHOUSHAN,XIANG SHAN
2,fd1d455b2c1d9607a678a5ec9e202b38,port_visit,2012-01-01 22:25:31,2012-01-21 16:04:45,28.1174,-15.3993,NaN,8364,ICCAT,"34, 34.1, 34.1.2",...,LAS PALMAS,0c40be0b,False,3.0,ESP,esp-laspalmas,28.079330,-15.386663,LAS PALMAS,LAS PALMAS
3,b59a7252bdb699af1c5d815197509df6,port_visit,2012-01-01 22:50:47,2012-01-06 11:23:26,53.1859,5.4000,"555514173, 555590821, 12754, 555542259, 478638...",5668,"NAMMCO, NASCO, IWC, ICCAT, ICES, ACAP, NEAFC","27, 27.4.c, 27.4",...,HARLINGEN,47c8ddd1,True,0.0,NLD,nld-harlingen,53.173996,5.410057,HARLINGEN,HARLINGEN
4,cac5d73ff8a2c876b0c5d1de48619dc6,port_visit,2012-01-01 22:58:01,2012-01-04 13:47:50,53.8856,8.7045,"555690907, 555517992",5669,"NAMMCO, ICCAT, NASCO, NEAFC, ICES, ACAP, IWC","27, 27.4, 27.4.b",...,CUXHAVEN,47b40415,True,0.0,DEU,deu-cuxhaven,53.874834,8.706188,CUXHAVEN,CUXHAVEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,c1edce0c75c71345f3ebc1a2854c3531,port_visit,2022-10-29 17:54:24,2022-10-30 03:55:03,68.4189,16.0160,NaN,5686,"NEAFC, IWC, ICCAT, ACAP, ICES, NAMMCO, NASCO","27.2.a, 27.2, 27, 27.2.a.2",...,FINNSNES,45dc11ef,True,0.0,NOR,nor-lodingen,68.413357,16.001389,LODINGEN,VESTFJORDEN
9943,111155427a347ffd150ef4657d766023,port_visit,2022-10-29 23:14:59,2022-10-30 21:43:18,5.2531,-4.0171,NaN,8473,"NAMMCO, IWC, ICCAT, ACAP","34.3.4, 34.3, 34",...,ABIDJAN,0fc1e7df,False,4.0,CIV,civ-abidjan,5.211140,-4.046577,ABIDJAN,ABIDJAN
9944,18728c95c3a39b95127b51af5ff28c04,port_visit,2022-10-30 03:58:48,2022-10-30 08:10:57,-43.1398,-73.6018,NaN,8465,"IWC, IATTC, ACAP, CPPS","87, 87.3, 87.3.1, 87.3.11",...,QUELLON,9621d18f,False,1.0,CHL,chl-quellon,-43.128987,-73.620161,QUELLON,QUELLON
9945,b42fb5657d1f991f437203f1dba38aaa,port_visit,2022-10-30 11:42:06,2022-10-30 11:59:07,38.1951,26.7835,NaN,5697,"CACFISH, EIFAAC","37, 37.3, 37.3.1",...,SIGACIK,14bb8cc5,True,0.0,TUR,tur-sigacik,38.194589,26.782568,SIGACIK,SIGACIK


In [68]:
port_visits_new = port_visits_new.drop_duplicates(subset=['event_id'])

In [74]:
port_visits_new.columns

Index(['event_id', 'event_type', 'event_start', 'event_end', 'event_lat',
       'event_lon', 'mpa', 'eez', 'rfmo', 'fao', 'major_fao', 'eez12nm',
       'boundingBox', 'startDistanceFromShoreKm', 'endDistanceFromShoreKm',
       'startDistanceFromPortKm', 'endDistanceFromPortKm', 'vessel_id',
       'vessel_flag', 'vessel_name', 'vessel_type', 'vessel_ssvid', 'visitId',
       'confidence', 'durationHrs', 'anchorage_id', 'at_dock',
       'distance_from_shore_km', 'flag', 'id', 'lat', 'lon', 'name',
       'top_destination', 'anchorage_id1', 'at_dock1',
       'distance_from_shore_km1', 'flag1', 'id1', 'lat1', 'lon1', 'name1',
       'top_destination1', 'anchorage_id2', 'at_dock2',
       'distance_from_shore_km2', 'flag2', 'id2', 'lat2', 'lon2', 'name2',
       'top_destination2'],
      dtype='object')

In [71]:
port_visits.columns

Index(['id', 'type', 'start', 'end', 'lat', 'lon', 'vessel.id', 'vessel.type',
       'vessel.mmsi', 'vessel.name', 'vessel.flag',
       'vessel.origin_port.country', 'vessel.origin_port.name',
       'vessel.destination_port.country', 'vessel.destination_port.name',
       'vessel.authorizations.authorized', 'vessel.authorizations.unknown',
       'median_speed_knots', 'elevation_m', 'distance_from_shore_m',
       'distance_from_port_m', 'regions.rfmo', 'encounter.median_distance_km',
       'encounter.authorization_status', 'encounter.encountered_vessel.id',
       'encounter.encountered_vessel.type',
       'encounter.encountered_vessel.mmsi',
       'encounter.encountered_vessel.name',
       'encounter.encountered_vessel.flag',
       'encounter.encountered_vessel.origin_port.country',
       'encounter.encountered_vessel.origin_port.name',
       'encounter.encountered_vessel.destination_port.country',
       'encounter.encountered_vessel.destination_port.name',
       'encount

In [30]:
port_visits = port_visits[['id', 'type', 'start', 'end', 'lat', 'lon', 'vessel.id', 'vessel.type',
       'vessel.mmsi', 'vessel.name', 'vessel.flag', 'port.lat', 'port.lon',
       'port.country', 'port.name']]
port_visits.columns

Index(['id', 'type', 'start', 'end', 'lat', 'lon', 'vessel.id', 'vessel.type',
       'vessel.mmsi', 'vessel.name', 'vessel.flag', 'port.lat', 'port.lon',
       'port.country', 'port.name'],
      dtype='object')

In [75]:
port_visits.columns = ['event_id', 'event_type', 'event_start', 'event_end', 'event_lat',
       'event_lon','vessel_id', 'vessel_type','vessel_ssvid','vessel_name', 'vessel_flag', 'lat','lon','flag','name']

In [76]:
port_visits2 = port_visits[~port_visits['event_id'].isin(port_visits_new['event_id'])]
port_visits2

,event_id,event_type,event_start,event_end,event_lat,event_lon,vessel_id,vessel_type,vessel_ssvid,vessel_name,vessel_flag,lat,lon,flag,name
0,af03d0b85979b8c91ae5ea165d0fd2e0,port,2012-10-24 12:21:05 UTC,2012-10-24 21:07:44 UTC,18.460661,-66.111945,3fe8729f8-80c4-2d9b-70cb-02aa09353f4e,carrier,311295000,DOLE ASIA,BHS,18.460661,-66.111945,PRI,SAN JUAN
1,76c123fc888c38f8424d444308e74739,port,2012-10-24 10:00:48 UTC,2012-10-26 14:49:48 UTC,8.024300,-76.780133,b3089e7df-fd55-d6e8-2292-e7bb093023fe,carrier,308090000,HELLAS STREAM,BHS,8.024300,-76.780133,COL,TURBO
2,a3910ba2f8e734f7343fe2c3886e031a,port,2012-10-24 23:02:21 UTC,2012-10-25 16:23:59 UTC,62.777300,6.917326,1c3ee5f35-5189-56ac-07a5-a99a6624b7aa,carrier,311411000,SAMSKIP ICE,BHS,62.777300,6.917326,NOR,NaN
3,c3bf496236204dead751072c35d5fefc,port,2012-10-24 14:57:38 UTC,2012-10-25 16:49:53 UTC,-16.990203,-72.142333,65032eb47-732f-946e-ea32-03567b0cefd8,carrier,636011106,BALTIC PRIDE,LBR,-16.990203,-72.142333,PER,MATARANI
4,ef23a729101f8d93679a51697754c7c3,port,2012-10-24 07:15:58 UTC,2012-10-25 09:39:17 UTC,52.874624,158.663179,6fc826893-3007-30a2-8cb3-dea6f5087bc0,carrier,273845620,POGRANICHNIK_ZMEEV,RUS,52.874624,158.663179,RUS,PETROPAVLOVSK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643485,eccf74a3775010a62c7ef191e782a7c9,port,2014-01-04 00:53:46 UTC,2014-01-05 05:03:33 UTC,51.298117,4.315254,f3bef3860-009c-e707-8995-895e9942e745,carrier,566060000,STAR ENDEAVOUR I,SGP,51.298117,4.315254,BEL,ANTWERP
643486,8c1808a4e82bac8d3f8c0db47d38fee8,port,2014-01-04 19:59:22 UTC,2014-01-05 05:12:58 UTC,53.978864,-166.412531,b59de4765-59b0-b4bf-b763-1ab505626338,carrier,368313000,PACIFIC VENTURE,USA,53.978864,-166.412531,USA,DUTCH HARBOR
643487,007253fe7ffa79318ff1ab60ccff4fd0,port,2014-01-04 21:10:29 UTC,2014-01-05 22:10:48 UTC,53.934971,-166.492739,f2b87e8b9-994a-d4f5-43f0-2603490b5e2d,carrier,367151060,KEVLEEN K,USA,53.934971,-166.492739,USA,DUTCH HARBOR
643488,fcec23ab8f3892b6d258d6c5ca4ca558,port,2014-01-04 13:26:44 UTC,2014-01-14 03:38:46 UTC,45.287153,14.406227,dd8ce3711-1009-7679-24b1-4582c408ec4c,carrier,314233000,BOSSE,BRB,45.287153,14.406227,HRV,RIJEKA LUKA


In [77]:
framesvisit = [port_visits2,port_visits_new]

resultvisit = pd.concat(framesvisit)
resultvisit

/Users/jinyingzhe/opt/anaconda3/lib/python3.9/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/Users/jinyingzhe/opt/anaconda3/lib/python3.9/site-packages/IPython/core/formatters.py:343: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,event_id,event_type,event_start,event_end,event_lat,event_lon,vessel_id,vessel_type,vessel_ssvid,vessel_name,...,top_destination1,anchorage_id2,at_dock2,distance_from_shore_km2,flag2,id2,lat2,lon2,name2,top_destination2
0,af03d0b85979b8c91ae5ea165d0fd2e0,port,2012-10-24 12:21:05 UTC,2012-10-24 21:07:44 UTC,18.460661,-66.111945,3fe8729f8-80c4-2d9b-70cb-02aa09353f4e,carrier,311295000,DOLE ASIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,76c123fc888c38f8424d444308e74739,port,2012-10-24 10:00:48 UTC,2012-10-26 14:49:48 UTC,8.024300,-76.780133,b3089e7df-fd55-d6e8-2292-e7bb093023fe,carrier,308090000,HELLAS STREAM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a3910ba2f8e734f7343fe2c3886e031a,port,2012-10-24 23:02:21 UTC,2012-10-25 16:23:59 UTC,62.777300,6.917326,1c3ee5f35-5189-56ac-07a5-a99a6624b7aa,carrier,311411000,SAMSKIP ICE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c3bf496236204dead751072c35d5fefc,port,2012-10-24 14:57:38 UTC,2012-10-25 16:49:53 UTC,-16.990203,-72.142333,65032eb47-732f-946e-ea32-03567b0cefd8,carrier,636011106,BALTIC PRIDE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ef23a729101f8d93679a51697754c7c3,port,2012-10-24 07:15:58 UTC,2012-10-25 09:39:17 UTC,52.874624,158.663179,6fc826893-3007-30a2-8cb3-dea6f5087bc0,carrier,273845620,POGRANICHNIK_ZMEEV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,c1edce0c75c71345f3ebc1a2854c3531,port_visit,2022-10-29 17:54:24,2022-10-30 03:55:03,68.418900,16.016000,d3c33831c-c587-bf0b-50e0-2a41f849896d,carrier,231766000,SILVER PEARL,...,FINNSNES,45dc11ef,True,0.0,NOR,nor-lodingen,68.413357,16.001389,LODINGEN,VESTFJORDEN
9943,111155427a347ffd150ef4657d766023,port_visit,2022-10-29 23:14:59,2022-10-30 21:43:18,5.253100,-4.017100,4b3eb996d-d6a1-a914-61e3-2151c105a03c,carrier,311001171,NOVA ZEELANDIA,...,ABIDJAN,0fc1e7df,False,4.0,CIV,civ-abidjan,5.211140,-4.046577,ABIDJAN,ABIDJAN
9944,18728c95c3a39b95127b51af5ff28c04,port_visit,2022-10-30 03:58:48,2022-10-30 08:10:57,-43.139800,-73.601800,b324f0d9a-a3a5-8c88-0103-45baa0fc387f,carrier,725003121,PATAGON IX,...,QUELLON,9621d18f,False,1.0,CHL,chl-quellon,-43.128987,-73.620161,QUELLON,QUELLON
9945,b42fb5657d1f991f437203f1dba38aaa,port_visit,2022-10-30 11:42:06,2022-10-30 11:59:07,38.195100,26.783500,04640cf01-19a7-ffa7-9d98-3ad99e550408,carrier,271072769,BASARANLAR,...,SIGACIK,14bb8cc5,True,0.0,TUR,tur-sigacik,38.194589,26.782568,SIGACIK,SIGACIK


In [178]:
compression_opts3 = dict(method='zip',
                        archive_name='new_carrier_port_visit.csv')  
resultvisit.to_csv('new_carrier_port_visit_merge.zip', index=False,
          compression=compression_opts3) 

In [112]:
carrier_port_visit['event_id'].isin(carrier_port_visit2['id']).sum()

18278

In [113]:
carrier_port_visit2

,id,type,start,end,lat,lon,vessel.id,vessel.type,vessel.mmsi,vessel.name,...,encounter.encountered_vessel.destination_port.country,encounter.encountered_vessel.destination_port.name,encounter.encountered_vessel.authorizations.authorized,encounter.encountered_vessel.authorizations.unknown,loitering.total_distance_km,loitering.loitering_hours,port.lat,port.lon,port.country,port.name
0,af03d0b85979b8c91ae5ea165d0fd2e0,port,2012-10-24 12:21:05 UTC,2012-10-24 21:07:44 UTC,18.460661,-66.111945,3fe8729f8-80c4-2d9b-70cb-02aa09353f4e,carrier,311295000,DOLE ASIA,...,NaN,NaN,NaN,NaN,NaN,NaN,18.460661,-66.111945,PRI,SAN JUAN
1,76c123fc888c38f8424d444308e74739,port,2012-10-24 10:00:48 UTC,2012-10-26 14:49:48 UTC,8.024300,-76.780133,b3089e7df-fd55-d6e8-2292-e7bb093023fe,carrier,308090000,HELLAS STREAM,...,NaN,NaN,NaN,NaN,NaN,NaN,8.024300,-76.780133,COL,TURBO
2,a3910ba2f8e734f7343fe2c3886e031a,port,2012-10-24 23:02:21 UTC,2012-10-25 16:23:59 UTC,62.777300,6.917326,1c3ee5f35-5189-56ac-07a5-a99a6624b7aa,carrier,311411000,SAMSKIP ICE,...,NaN,NaN,NaN,NaN,NaN,NaN,62.777300,6.917326,NOR,NaN
3,c3bf496236204dead751072c35d5fefc,port,2012-10-24 14:57:38 UTC,2012-10-25 16:49:53 UTC,-16.990203,-72.142333,65032eb47-732f-946e-ea32-03567b0cefd8,carrier,636011106,BALTIC PRIDE,...,NaN,NaN,NaN,NaN,NaN,NaN,-16.990203,-72.142333,PER,MATARANI
4,ef23a729101f8d93679a51697754c7c3,port,2012-10-24 07:15:58 UTC,2012-10-25 09:39:17 UTC,52.874624,158.663179,6fc826893-3007-30a2-8cb3-dea6f5087bc0,carrier,273845620,POGRANICHNIK_ZMEEV,...,NaN,NaN,NaN,NaN,NaN,NaN,52.874624,158.663179,RUS,PETROPAVLOVSK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643485,eccf74a3775010a62c7ef191e782a7c9,port,2014-01-04 00:53:46 UTC,2014-01-05 05:03:33 UTC,51.298117,4.315254,f3bef3860-009c-e707-8995-895e9942e745,carrier,566060000,STAR ENDEAVOUR I,...,NaN,NaN,NaN,NaN,NaN,NaN,51.298117,4.315254,BEL,ANTWERP
643486,8c1808a4e82bac8d3f8c0db47d38fee8,port,2014-01-04 19:59:22 UTC,2014-01-05 05:12:58 UTC,53.978864,-166.412531,b59de4765-59b0-b4bf-b763-1ab505626338,carrier,368313000,PACIFIC VENTURE,...,NaN,NaN,NaN,NaN,NaN,NaN,53.978864,-166.412531,USA,DUTCH HARBOR
643487,007253fe7ffa79318ff1ab60ccff4fd0,port,2014-01-04 21:10:29 UTC,2014-01-05 22:10:48 UTC,53.934971,-166.492739,f2b87e8b9-994a-d4f5-43f0-2603490b5e2d,carrier,367151060,KEVLEEN K,...,NaN,NaN,NaN,NaN,NaN,NaN,53.934971,-166.492739,USA,DUTCH HARBOR
643488,fcec23ab8f3892b6d258d6c5ca4ca558,port,2014-01-04 13:26:44 UTC,2014-01-14 03:38:46 UTC,45.287153,14.406227,dd8ce3711-1009-7679-24b1-4582c408ec4c,carrier,314233000,BOSSE,...,NaN,NaN,NaN,NaN,NaN,NaN,45.287153,14.406227,HRV,RIJEKA LUKA


In [79]:
carriers_new['mmsi']

0       309361000
1       538003418
2       263673930
3       538002995
4       356386000
          ...    
4630    311001077
4631    525300575
4632    367019340
4633    525300999
4634    306873000
Name: mmsi, Length: 4635, dtype: int64

In [90]:
resultvisit['vessel_ssvid'][~resultvisit['vessel_ssvid'].isin(carriers_new['mmsi'])].unique().shape

(3864,)

In [92]:
resultvisit['vessel_ssvid'].unique().shape

(5871,)

In [94]:
resultvisit['vessel_ssvid'][resultvisit['vessel_ssvid'].isin(carriers_new['mmsi'])].unique().shape

(2007,)

In [100]:
result['vessel.mmsi'].unique()

array([636015528, 370070000, 356591000, 412425561, 372907000, 273358410,
       636009737, 368076750, 372886000, 416308000, 577106000, 367408050,
       366841470, 366588000, 271072769, 412440834, 273376120, 372942000,
       636010125, 367041530, 636017161, 214182223, 273251100, 353937000,
       412203740, 412440514, 725001544, 273440860, 273875400, 273384570,
       367119230, 341049000, 273397490, 353079000, 273219900, 367041640,
       576507000, 367098030, 353306000, 371630000, 273352570, 273252100,
       367724590, 273447930, 412421071, 412440723, 636018227, 636013651,
       412331208, 413692650, 636011004, 273385170, 412420808, 374906000,
       441923000, 367016340, 273420160, 412549527, 374173000, 273215390,
       273825810, 371275000, 431471000, 412549349, 357642000, 725003970,
       372033000, 351350000, 355966000, 312255000, 431600125, 224434000,
       273219320, 273330280, 412423655, 367113130, 431601458, 412420998,
       576328000, 416521000, 367510750, 271073249, 

In [101]:
result3['vessel.mmsi'].unique()

array([416308000, 577106000, 352894000, ..., 671904000, 257307000,
       647801301])

In [102]:
result

,id,type,start,end,lat,lon,vessel.id,vessel.type,vessel.mmsi,vessel.name,...,major_fao,eez12nm,boundingBox,startDistanceFromShoreKm,endDistanceFromShoreKm,startDistanceFromPortKm,endDistanceFromPortKm,medianDistanceKilometers,medianSpeedKnots,encounter_type
0,97e6c194a35b0c7f8ae8a98ed7b03343.1,encounter,2015-02-19 13:20:00,2015-02-19 23:50:00,-45.535014,-60.491390,000702b7e-e442-5969-7849-35ea992140af,carrier,636015528,PRINCE OF STREAM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9c553adb149c1899849dd46576b1442c.1,encounter,2015-02-17 15:30:00,2015-02-17 23:50:00,-45.554551,-60.497431,000702b7e-e442-5969-7849-35ea992140af,carrier,636015528,PRINCE OF STREAM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d85f6e4b3a150ca65360a1baa7596a8f.1,encounter,2015-02-15 17:40:00,2015-02-15 23:50:00,-45.554480,-60.494979,000702b7e-e442-5969-7849-35ea992140af,carrier,636015528,PRINCE OF STREAM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,be1c9528d50b7fb1ac5fbfb3bc55ee10.1,encounter,2015-05-30 00:00:00,2015-05-30 08:30:00,-45.505639,-60.486418,000702b7e-e442-5969-7849-35ea992140af,carrier,636015528,PRINCE OF STREAM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,61a91a71ed5d98ee8d34f84a3765bc31.1,encounter,2015-02-02 20:10:00,2015-02-03 06:20:00,-45.583149,-60.494116,000702b7e-e442-5969-7849-35ea992140af,carrier,636015528,PRINCE OF STREAM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118289,a65130c8f7d4ec91cba11538f370cdf4.2,encounter,2022-10-30 16:50:00,2022-10-30 19:00:00,59.197100,163.333600,c600c9133-3e3e-a863-7f7a-fd9821b49db3,carrier,273421150,SVYATOY VLADIMIR,...,61.0,NaN,"163.3336104245, 59.1970912283, 163.3336104245,...",5.0,5.0,13.285359,13.285359,0.125,0.083,carrier-fishing
118291,9c0c94c12a4614399b396aaed95ea500.1,encounter,2022-10-30 18:20:00,2022-10-30 21:40:00,-17.767200,-81.112400,0bc5c51c9-9c68-0015-aa73-f3f459a498be,carrier,636018227,WEI NING,...,87.0,NaN,"-81.1123629956, -17.76723656205, -81.112362995...",651.0,651.0,678.767375,678.767375,0.040,1.063,carrier-fishing
118292,415d1e7cf9583935b5262527321ec423.2,encounter,2022-10-30 19:00:00,2022-10-30 23:50:00,59.517200,163.252200,e6e8bbf73-3fbd-dc7a-f4cd-32f9c4ace74e,carrier,273385860,SLAVYANKA,...,61.0,NaN,"163.2521519741, 59.51719611865, 163.2521519741...",2.0,2.0,31.201092,31.201092,0.029,0.053,carrier-fishing
118294,acb49a8ab2d13872cce1cd3c949a8726.2,encounter,2022-10-30 19:00:00,2022-10-30 23:50:00,-1.982100,-150.084500,6ffc1783d-de5b-56d5-0706-148f072af8df,carrier,440237000,OCEANUS,...,77.0,NaN,"-150.08448811675, -1.9820948907, -150.08448811...",581.0,581.0,933.044375,933.044375,0.048,0.948,carrier-fishing


In [120]:
result.columns

Index(['id', 'type', 'start', 'end', 'lat', 'lon', 'vessel.id', 'vessel.type',
       'vessel.mmsi', 'vessel.name', 'vessel.flag',
       'vessel.origin_port.country', 'vessel.origin_port.name',
       'vessel.destination_port.country', 'vessel.destination_port.name',
       'vessel.authorizations.authorized', 'vessel.authorizations.unknown',
       'median_speed_knots', 'elevation_m', 'distance_from_shore_m',
       'distance_from_port_m', 'regions.rfmo', 'encounter.median_distance_km',
       'encounter.authorization_status', 'encounter.encountered_vessel.id',
       'encounter.encountered_vessel.type',
       'encounter.encountered_vessel.mmsi',
       'encounter.encountered_vessel.name',
       'encounter.encountered_vessel.flag',
       'encounter.encountered_vessel.origin_port.country',
       'encounter.encountered_vessel.origin_port.name',
       'encounter.encountered_vessel.destination_port.country',
       'encounter.encountered_vessel.destination_port.name',
       'encount

In [137]:
carrier_from_loitering = result3[['vessel.id', 'vessel.type','vessel.mmsi', 'vessel.name', 'vessel.flag']].copy()
carrier_from_loitering = carrier_from_loitering[carrier_from_loitering['vessel.type']=='carrier']
carrier_from_loitering.columns = ['id', 'vesselType', 'mmsi', 'shipname', 'flag']
carrier_from_loitering = carrier_from_loitering.drop_duplicates(subset=['mmsi'])
carrier_from_loitering = carrier_from_loitering[~carrier_from_loitering['mmsi'].isin(carriers_new['mmsi'])]
carrier_from_loitering['dataset'] = 'API-loitering'
frames_carr_loi = [carriers_new,carrier_from_loitering]
carrier1 = pd.concat(frames_carr_loi)

In [139]:
carrier_from_encounters = result[['vessel.id', 'vessel.type','vessel.mmsi', 'vessel.name', 'vessel.flag']].copy()
carrier_from_encounters = carrier_from_encounters[carrier_from_encounters['vessel.type']=='carrier']
carrier_from_encounters.columns = ['id', 'vesselType', 'mmsi', 'shipname', 'flag']
carrier_from_encounters = carrier_from_encounters.drop_duplicates(subset=['mmsi'])
carrier_from_encounters = carrier_from_encounters[~carrier_from_encounters['mmsi'].isin(carriers_new['mmsi'])]
carrier_from_encounters['dataset'] = 'API-encounters'
frames_carr_enc = [carrier1,carrier_from_encounters]
carrier2 = pd.concat(frames_carr_enc)

In [142]:
carrier_from_port = resultvisit[['vessel_id','vessel_type','vessel_ssvid','vessel_name','vessel_flag']].copy()
carrier_from_port = carrier_from_port[carrier_from_port['vessel_type']=='carrier']
carrier_from_port.columns = ['id', 'vesselType', 'mmsi', 'shipname', 'flag']
carrier_from_port = carrier_from_port.drop_duplicates(subset=['mmsi'])
carrier_from_port = carrier_from_port[~carrier_from_port['mmsi'].isin(carriers_new['mmsi'])]
carrier_from_port['dataset'] = 'API-port'
frames_carr_por = [carrier2,carrier_from_port]
carrier3 = pd.concat(frames_carr_por)

In [143]:
carrier3

,name,callsign,firstTransmissionDate,flag,geartype,id,imo,lastTransmissionDate,mmsi,msgCount,posCount,shipname,source,vesselType,years,dataset,score
0,1.0,C6VZ4,2019-10-08T08:20:23Z,BHS,NaN,7c0e13c65-5fac-3e25-2472-3629a4dbfbb4,8922242.0,2020-07-05T17:08:38Z,309361000,7979742.0,79325.0,GREEN EXPLORER,AIS,Carrier,"2019, 2020",public-global-carrier-vessels:v20201001,1.0
1,2.0,V7RC6,2012-01-04T18:18:12Z,MHL,NaN,863ec9582-273e-0682-b3f3-80b03e07614a,8617926.0,2017-06-23T11:30:02Z,538003418,4949842.0,123069.0,PACIFIC BREEZE,AIS,Carrier,"2012, 2013, 2014, 2015, 2016, 2017",public-global-carrier-vessels:v20201001,1.0
2,3.0,CSZF2,2019-05-15T14:54:31Z,PRT,NaN,32dd2dad6-6990-f13f-4397-77c2c08c912b,8541816.0,2022-11-01T23:54:22Z,263673930,49245098.0,210312.0,RABILHO,AIS,Carrier,"2019, 2020, 2021, 2022",public-global-carrier-vessels:v20201001,1.0
3,4.0,V7NP7,2012-01-03T16:42:49Z,MHL,NaN,bfbb22402-2837-5ba6-5e58-7b703b44946a,8810645.0,2013-07-19T02:51:16Z,538002995,1426660.0,16174.0,KHANA,AIS,Carrier,"2012, 2013",public-global-carrier-vessels:v20201001,1.0
4,5.0,HOZZ,2012-01-03T17:16:26Z,PAN,NaN,ecda1c1d6-6dbd-1842-b372-27ddc1682e09,9000376.0,2022-11-01T23:51:22Z,356386000,64292216.0,875014.0,SUAH,AIS,Carrier,"2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019...",public-global-carrier-vessels:v20201001,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622557,NaN,NaN,NaN,CHN,NaN,fb478ed09-93e5-ac70-def4-ae5974227ec2,NaN,NaN,412549394,NaN,NaN,FU YUAN YU 8654,NaN,carrier,NaN,API-port,NaN
622574,NaN,NaN,NaN,CHN,NaN,ac68d2723-3b7f-3be3-d92c-a4cc7878c024,NaN,NaN,412549391,NaN,NaN,FU YUAN YU 8651,NaN,carrier,NaN,API-port,NaN
622635,NaN,NaN,NaN,CHN,NaN,74551b9c6-616e-fda7-bf47-7dc06f34e7d9,NaN,NaN,412549392,NaN,NaN,FU YUAN YU 8652,NaN,carrier,NaN,API-port,NaN
624768,NaN,NaN,NaN,CHN,NaN,669043fc8-88d8-12dd-abda-4d9a5c047980,NaN,NaN,412421134,NaN,NaN,XIN SHI JI 65,NaN,carrier,NaN,API-port,NaN
